In [1]:
from h2_gym.envs import Planning
from h2_plan.data import DefaultParams

In [ ]:
environment = Planning('NH3-Chile', weather_file ='CoastalChile_15-20_Wind.csv')

with environment as env:
    env['booleans']['vector_choice']['NH3'] = True
    env['booleans']['electrolysers']['SOFC'] = True
    env['booleans']['wind'] = True
    env['equipment']['vector_production']['NH3'] = 5

environment.solve()

[INFO] Reading CSV file: /Users/cameronaldren/miniconda3/envs/h2gym/lib/python3.10/site-packages/meteor_py/data/CoastalChile_15-20_Wind.csv
[INFO] Setup Model completed in 0.07 seconds


In [ ]:
res = environment.get_results()


In [4]:
environment.visualise()

2025-05-23 13:45:28.556 python[28239:886780] +[IMKClient subclass]: chose IMKClient_Legacy
2025-05-23 13:45:28.556 python[28239:886780] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


: 

In [ ]:
import yaml
import os
from pathlib import Path

params = DefaultParams()
param_drops = [
    ['efficiencies','reconversion'],
    ['efficiencies','battery_storage'],
    ['capital_costs','battery_storage'],
    ['capital_costs','geographical_storage'],
    ['capital_costs','shipping'],
    ['capital_costs','reconversion'],
    ['capital_costs','port'],
    ['operating_costs','battery_storage'],
    ['operating_costs','geographical_storage'],
    ['operating_costs','shipping'],
    ['operating_costs','reconversion'],
    ['operating_costs','port'],
    ['replacement_frequencies','battery_storage'],
    ['replacement_frequencies','reconversion'],
    ['shipping','fuel_consumption'],
    ['shipping','journey_time'],
    ['shippping','charter_rate'],
    ['shipping','berthing_rate'],
    ['miscillaneous','reconversion_electrical_demand'],
    ['miscillaneous','battery_charge_time'],
    ['equipment','gh2_storage'],
    ['equipment','origin_storage'],
    ['equipment','capacity_electrolysers'],
    ['equipment','number_turbines'],
    ['equipment','compression_capacity'],
    ['equipment','hfc_capacity'],
]

bool_drops = [
    'net_present_value',
    'geographical_storage',
    'grid_wheel'
]

def recursive_drop(d, keys):
    """Recursively delete the value associated with a list of nested keys in dict d."""
    if len(keys) == 1:
        d.pop(keys[0], None)
    else:
        if keys[0] in d and isinstance(d[keys[0]], dict):
            recursive_drop(d[keys[0]], keys[1:])

for i in params.formulation_parameters:
    for rec_key in param_drops:
        recursive_drop(params.formulation_parameters, rec_key)
    

for j in environment._booleans:
    for rec_key in bool_drops:
        recursive_drop(environment._booleans, rec_key)
   
params.formulation_parameters['booleans'] = environment._booleans

# Ensure the directory exists
# In Jupyter notebooks, __file__ is not defined. Use the current working directory instead.
output_path = Path.cwd() / 'src' / 'h2-gym' / 'tmp' / 'params.yaml'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, 'w') as file:
    yaml.dump(params.formulation_parameters, file)

print(f"Parameters saved to {output_path}")

Parameters saved to /Users/cameronaldren/miniconda3/envs/h2gym/lib/python3.10/site-packages/h2_plan/opt/src/h2-gym/tmp/params.yaml
